In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [0]:
# Schema for read data in dataframe 
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)
])

In [0]:
circuits_df = spark.read \
.option("header", True) \
.schema(circuits_schema) \
.csv("/mnt/jdoformula1dl/raw/circuits.csv")

In [0]:
from pyspark.sql.functions import col

In [0]:
# Select only required columns
circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))

In [0]:
# Rename columns for better clarity
circuits_renamed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude")

In [0]:
# Add ingestion date column
from pyspark.sql.functions import current_timestamp
circuits_final_df = circuits_renamed_df.withColumn("ingestion_date", current_timestamp()) 

In [0]:
# Write data to datalake as parquet
circuits_final_df.write.mode("overwrite").parquet("/mnt/jdoformula1dl/processed/circuits")

In [0]:
display(spark.read.parquet("/mnt/jdoformula1dl/processed/circuits"))